# Шаблон для проекта по A/B - тестам.

### `chat_id ` из Телеграма:

In [26]:
chat_id = 460109099

------------------

##  <ins>Задание 1.</ins>

### 0. Запосить у бота в Телеграме условие командой `/get_project1_desc`.

### 1. Необходимо выбрать метрику и привести аргументацию.

Так как данные по среднему доходу ждать нет времени, можно отслеживать только сам факт успешности звонков, т.е. конверсию из дозвона в продажу. Это и будет целевой метрикой.

In [11]:
import pandas as pd
data1 = pd.read_csv('hist_telesales.csv').drop(columns='ID')

In [12]:
data1.head(20)

,Флаг дозвона,Флаг продажи,Расходы,PV,NPV
0,1,0,90,0,-90
1,0,0,5,0,-5
2,0,0,68,0,-68
3,1,0,22,0,-22
4,1,0,22,0,-22
5,1,1,367,2397,2030
6,1,1,49,1213,1164
7,0,0,26,0,-26
8,0,0,35,0,-35
9,0,0,61,0,-61


In [16]:
conv = data1[data1["Флаг дозвона"]==1]["Флаг продажи"].mean()
print('конверсия из дозвона в продажу:', conv)
uplift = 0.08
MDE = conv * uplift
print('\nMDE:', MDE)

конверсия из дозвона в продажу: 0.503719275549806

MDE: 0.04029754204398448


### 2. Альтернатива в критерии.

Выбираем двухстороннюю альтернативу с гипотезой о равенстве средних. Если по критерию средние не совпадут, посмотрим на конверсии и сделаем выводы.

### 3.1. Каков размер выборки? Привести аргументацию и написать как получилось то или иное число.

In [19]:
from scipy.stats import norm

k = 0.5 # разбитие потока равновероятно
alpha = 0.05 # уровень значимости
beta = 0.8 # уверенность в различии изменений целевой метрики

Узнаем размер выборки теста по формуле (для двухсторонней альтернативы):

In [25]:
n = conv*(1-conv) * ( (norm.ppf(alpha/2) - norm.ppf(beta))**2 )/( 0.5*MDE**2 )
print('\nРазмер выборки:', int(n))


Размер выборки: 2416


### 3.2. Необходимо запросить у бота в Телеграме выборку полученного размера командой `/get_project1_sample {размер выборки}` и вписать <ins>код выборки</ins>:

In [24]:
38277

38277

In [42]:
control1 = pd.read_csv("Контроль.csv").drop(columns="ID")
test1 = pd.read_csv("Тест.csv").drop(columns="ID")

### 4. Принятие решения. Расписать подробно с аргументами.

In [30]:
from statsmodels.stats.proportion import proportions_ztest

In [43]:
count_control1 = control1[control1["Флаг дозвона"]==1]["Флаг продажи"].sum()
count_test1 = test1[test1["Флаг дозвона"]==1]["Флаг продажи"].sum()
nobs_control1 = len(control1[control1["Флаг дозвона"]==1])
nobs_test1 = len(test1[test1["Флаг дозвона"]==1])

_, p_value = proportions_ztest([count_control1, count_test1], [nobs_control1, nobs_test1], alternative = "two-sided")
print('p_value =', p_value)

p_value = 1.074048634319037e-05


Критерий отвергает гипотезу, посмотрим среднее по данным.

In [44]:
print('Контроль:', control1[control1["Флаг дозвона"]==1]["Флаг продажи"].mean())
print ('Тест:    ', test1[test1["Флаг дозвона"]==1]["Флаг продажи"].mean())

Контроль: 0.5249146757679181
Тест:     0.44360902255639095


Как видно, конверсия с жёстким скриптом больше, поэтому стоит оставить все как было.

------------------

##  <ins>Задание 2.</ins>

### 0. Запосить у бота в Телеграме условие командой `/get_project2_desc`.

### 1. Необходимо выбрать метрику и привести аргументацию.

### 2. Альтернатива в критерии.

### 3. Нужно выбрать параметр(-ы), влияющий(-ие) на целевую метрику. Привести аргументацию.

### 4.1. Каков размер выборки? Привести аргументацию и написать как получилось то или иное число.

### 4.2. Необходимо запросить у бота в Телеграме выборку полученного размера командой `/get_project2_sample {размер выборки}` и вписать <ins>код выборки</ins>:

### 5. Проверка на однородность, применение критерия. Принятие решения. Расписать подробно с аргументами.